The "Semi-ATE/STDF" library is designed for parsing and working with STDF files in Python, and it aims to convert STDF data into pandas DataFrames for easier analysis. While the library itself is excellent for reading and manipulating STDF data, it doesn't have a direct, built-in function like `merge_and_keep_final_results` that handles the specific logic of merging multiple STDF files from the same lot, where later files represent re-tests and you only want the *final* result for each part/test.

This kind of merging requires custom logic because the STDF format is a stream of records, and identifying "redundant" tests and "final" results for a specific part/test combination requires understanding the test flow and potentially tracking part IDs and test numbers across multiple files.

Here's a conceptual outline and a Python code example using `Semi-ATE/STDF` and `pandas` to achieve your goal. This approach assumes:

1.  **Unique Part Identification:** Each part tested has a unique identifier (e.g., in the PRR record).
2.  **Test Identification:** Each test within a part's sequence can be uniquely identified (e.g., by test number, test name, or a combination).
3.  **Retest Logic:** A later test for the same part/test combination in a subsequent file supersedes previous results. The order of files matters: the last file processed for a given part/test combination is considered the "final" result.

**Core Idea:**

1.  **Parse each STDF file:** Convert the relevant records (like Part Results (PRR), Test Results (PTR), Site Results (STR), etc.) into pandas DataFrames.
2.  **Identify relevant fields:** Determine which columns in your DataFrames uniquely identify a part (e.g., `HEAD_NUM`, `SITE_NUM`, `PART_ID`) and a specific test within that part (e.g., `TEST_NUM`, `TEST_NAM`).
3.  **Merge DataFrames:** Combine the DataFrames from all STDF files.
4.  **Deduplicate and keep final results:** Use a combination of sorting and `drop_duplicates` to keep only the latest (final) result for each part-test combination.

**Example Python Code:**

First, ensure you have the `Semi-ATE` and `pandas` libraries installed:

```bash
pip install Semi-ATE pandas
```

Now, here's the Python code:

In [ ]:
import os
import pandas as pd
from Semi_ATE.STDF import V4

def parse_stdf_to_dataframe(stdf_file_path):
    """
    Parses an STDF file and extracts relevant records into a pandas DataFrame.
    Focuses on PTR records for test results.
    You might need to adjust this based on what records you need.
    """
    data = []
    try:
        with open(stdf_file_path, 'rb') as f:
            for record in V4.records_from_file(f):
                if isinstance(record, V4.PTR):
                    # Extract key information from PTR record
                    data.append({
                        'FILE_PATH': stdf_file_path, # Keep track of the source file
                        'REC_TYP': record.REC_TYP,
                        'REC_SUB': record.REC_SUB,
                        'HEAD_NUM': record.HEAD_NUM,
                        'SITE_NUM': record.SITE_NUM,
                        'TEST_NUM': record.TEST_NUM,
                        'TEST_NAM': record.TEST_NAM if hasattr(record, 'TEST_NAM') else None,
                        'HARD_BIN': record.HARD_BIN if hasattr(record, 'HARD_BIN') else None,
                        'SOFT_BIN': record.SOFT_BIN if hasattr(record, 'SOFT_BIN') else None,
                        'RESULT': record.RESULT, # The test result (e.g., value, pass/fail code)
                        'TEST_FLG': record.TEST_FLG, # Test flags (e.g., pass/fail status)
                        'RTN_ICNT': record.RTN_ICNT, # Retest count - very important for your case
                        'X_COORD': record.X_COORD if hasattr(record, 'X_COORD') else None,
                        'Y_COORD': record.Y_COORD if hasattr(record, 'Y_COORD') else None,
                        # Add other relevant fields from PTR or other records like PRR if needed
                        # For PRR (Part Results Record), you might also want to track PART_ID
                        # To get PART_ID, you'd need to link PRR records to PTRs.
                        # This example simplifies by only getting PTRs.
                    })
                # You might need to process PRR records to get PART_ID
                # For simplicity, we'll assume HEAD_NUM and SITE_NUM are enough for part identification for this example.
                # If PART_ID is crucial, you'll need to store PRR data and merge it based on sequence.
                # Or, if your PTRs contain unique part IDs, use that.
    except Exception as e:
        print(f"Error processing {stdf_file_path}: {e}")
    return pd.DataFrame(data)

def merge_stdf_files_final_results(stdf_files_paths, output_csv_path="merged_final_results.csv"):
    """
    Merges multiple STDF files, keeping only the final test result for each
    part/test combination.

    Args:
        stdf_files_paths (list): A list of paths to the STDF files, ordered
                                 from oldest to newest (first is largest, last is smallest).
        output_csv_path (str): The path to save the merged results as a CSV.
    """
    all_dataframes = []

    print("Parsing STDF files...")
    for file_path in stdf_files_paths:
        print(f"  - Parsing {file_path}")
        df = parse_stdf_to_dataframe(file_path)
        if not df.empty:
            df['SOURCE_FILE_INDEX'] = stdf_files_paths.index(file_path) # Helps in ordering later
            all_dataframes.append(df)

    if not all_dataframes:
        print("No data parsed from any STDF files.")
        return

    # Concatenate all dataframes
    merged_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"Total records after initial merge: {len(merged_df)}")

    # Sort to ensure later files' results come after earlier ones for the same part/test.
    # The order of sorting keys is crucial for correct deduplication.
    # We sort by:
    # 1. Part identifier (HEAD_NUM, SITE_NUM - if PART_ID is available, use it!)
    # 2. Test identifier (TEST_NUM, TEST_NAM)
    # 3. Source file index (to prioritize later files in case of exact duplicates in earlier files)
    # 4. Retest count (RTN_ICNT) - higher retest count indicates a later run for the same part/test within a file or across files if the part was re-tested.
    merged_df.sort_values(
        by=['HEAD_NUM', 'SITE_NUM', 'TEST_NUM', 'TEST_NAM', 'SOURCE_FILE_INDEX', 'RTN_ICNT'],
        inplace=True,
        ascending=[True, True, True, True, True, True] # Ascending for all to ensure later tests are at the bottom
    )

    # Drop duplicates, keeping the last occurrence.
    # 'HEAD_NUM', 'SITE_NUM', 'TEST_NUM', 'TEST_NAM' define a unique test for a unique part.
    # Because of the sorting, `keep='last'` will select the entry from the latest file (highest SOURCE_FILE_INDEX)
    # and the highest RTN_ICNT for that part/test combination.
    final_results_df = merged_df.drop_duplicates(
        subset=['HEAD_NUM', 'SITE_NUM', 'TEST_NUM', 'TEST_NAM'],
        keep='last'
    )
    print(f"Total records after deduplication (final results): {len(final_results_df)}")

    # Clean up and save
    final_results_df = final_results_df.drop(columns=['FILE_PATH', 'SOURCE_FILE_INDEX'])
    final_results_df.to_csv(output_csv_path, index=False)
    print(f"Merged final results saved to {output_csv_path}")

# --- Example Usage ---
if __name__ == "__main__":
    # Create dummy STDF files for demonstration
    # In a real scenario, you would replace these with your actual file paths.

    # File 1 (largest): Initial run, some failures
    # File 2 (smaller): Retest of some failures from File 1
    # File 3 (smallest): Retest of some remaining failures from File 2

    # Dummy data for demonstration
    # For simplicity, we'll manually create some STDF records.
    # In a real scenario, these would be generated by your tester.

    # Helper to write dummy STDF files
    def write_dummy_stdf(file_path, records):
        with open(file_path, 'wb') as f:
            for rec in records:
                f.write(rec.to_bytes())

    # Dummy records for File 1
    # Part 1, Site 0: Test 1 (Pass), Test 2 (Fail)
    # Part 2, Site 1: Test 1 (Pass), Test 2 (Pass)
    dummy_records_1 = [
        V4.FAR(CPU_TYPE=2, STDF_VER=4),
        V4.MIR(LOT_ID='LOT_ABC', NODE_NAM='Tester1', TSTR_TYP='ATE', TEST_TIM=1678886400),
        V4.SDR(HEAD_NUM=1, SITE_GRP=0, SITE_NUM=1), # Dummy site information
        V4.PRR(HEAD_NUM=1, SITE_NUM=0, PART_FLG=0x00, NUM_TEST=2, HARD_BIN=1, SOFT_BIN=1), # Part 1 (Site 0)
        V4.PTR(HEAD_NUM=1, SITE_NUM=0, TEST_NUM=1, TEST_FLG=0x00, RESULT=10.5, RTN_ICNT=0), # Part 1 Test 1 (Pass)
        V4.PTR(HEAD_NUM=1, SITE_NUM=0, TEST_NUM=2, TEST_FLG=0x80, RESULT=1.2, RTN_ICNT=0), # Part 1 Test 2 (Fail)
        V4.PRR(HEAD_NUM=1, SITE_NUM=1, PART_FLG=0x00, NUM_TEST=2, HARD_BIN=1, SOFT_BIN=1), # Part 2 (Site 1)
        V4.PTR(HEAD_NUM=1, SITE_NUM=1, TEST_NUM=1, TEST_FLG=0x00, RESULT=20.1, RTN_ICNT=0), # Part 2 Test 1 (Pass)
        V4.PTR(HEAD_NUM=1, SITE_NUM=1, TEST_NUM=2, TEST_FLG=0x00, RESULT=5.0, RTN_ICNT=0),  # Part 2 Test 2 (Pass)
        V4.MRR(FINL_PMOD=0)
    ]
    stdf_file_1 = "lot_test_1.stdf"
    write_dummy_stdf(stdf_file_1, dummy_records_1)

    # Dummy records for File 2 (retest of Part 1, Test 2)
    # Part 1, Site 0: Test 2 (Pass) - this is the retest, so RTN_ICNT should be > 0 or it's a new run
    dummy_records_2 = [
        V4.FAR(CPU_TYPE=2, STDF_VER=4),
        V4.MIR(LOT_ID='LOT_ABC', NODE_NAM='Tester2', TSTR_TYP='ATE', TEST_TIM=1678886500),
        V4.SDR(HEAD_NUM=1, SITE_GRP=0, SITE_NUM=1),
        V4.PRR(HEAD_NUM=1, SITE_NUM=0, PART_FLG=0x00, NUM_TEST=1, HARD_BIN=1, SOFT_BIN=1), # Part 1 (Site 0)
        V4.PTR(HEAD_NUM=1, SITE_NUM=0, TEST_NUM=2, TEST_FLG=0x00, RESULT=1.5, RTN_ICNT=1), # Part 1 Test 2 (Retest, now Pass)
        V4.MRR(FINL_PMOD=0)
    ]
    stdf_file_2 = "lot_test_2.stdf"
    write_dummy_stdf(stdf_file_2, dummy_records_2)

    # Dummy records for File 3 (a different part retest)
    # Part 3, Site 0: Test 1 (Pass)
    dummy_records_3 = [
        V4.FAR(CPU_TYPE=2, STDF_VER=4),
        V4.MIR(LOT_ID='LOT_ABC', NODE_NAM='Tester3', TSTR_TYP='ATE', TEST_TIM=1678886600),
        V4.SDR(HEAD_NUM=1, SITE_GRP=0, SITE_NUM=1),
        V4.PRR(HEAD_NUM=1, SITE_NUM=2, PART_FLG=0x00, NUM_TEST=1, HARD_BIN=1, SOFT_BIN=1), # Part 3 (Site 2)
        V4.PTR(HEAD_NUM=1, SITE_NUM=2, TEST_NUM=1, TEST_FLG=0x00, RESULT=7.8, RTN_ICNT=0), # Part 3 Test 1 (Pass)
        V4.MRR(FINL_PMOD=0)
    ]
    stdf_file_3 = "lot_test_3.stdf"
    write_dummy_stdf(stdf_file_3, dummy_records_3)


    # List of STDF files in the order they were generated (or re-tested)
    # This order is crucial for correctly identifying "final" results.
    stdf_files = [stdf_file_1, stdf_file_2, stdf_file_3]
    output_csv = "merged_final_stdf_results.csv"

    merge_stdf_files_final_results(stdf_files, output_csv)

    print("\nContent of the merged CSV file:")
    print(pd.read_csv(output_csv))

    # Clean up dummy files
    os.remove(stdf_file_1)
    os.remove(stdf_file_2)
    os.remove(stdf_file_3)

**Explanation:**

1.  **`parse_stdf_to_dataframe(stdf_file_path)` function:**
    * This function takes an STDF file path as input.
    * It uses `V4.records_from_file()` from `Semi_ATE.STDF` to iterate through the records in the STDF file.
    * It specifically targets `V4.PTR` (Parametric Test Record) records, as these contain the actual test results. You might also want to extract data from `V4.PRR` (Part Results Record) if `PART_ID` is defined there and you need it for unique part identification.
    * For each `PTR` record, it extracts key information like `HEAD_NUM` (test head number), `SITE_NUM` (test site number), `TEST_NUM` (test number), `TEST_NAM` (test name), `RESULT`, `TEST_FLG` (pass/fail status), and crucially, `RTN_ICNT` (retest count).
    * `RTN_ICNT` is important as it indicates if a test for a particular part has been re-run. A higher `RTN_ICNT` generally implies a later retest.
    * It adds a `FILE_PATH` column to track the source file.
    * All extracted data is accumulated into a list of dictionaries, which is then converted into a pandas DataFrame.

2.  **`merge_stdf_files_final_results(stdf_files_paths, output_csv_path)` function:**
    * Takes a list of STDF file paths and an output CSV path.
    * **Parsing:** It iterates through each `stdf_file_path`, calls `parse_stdf_to_dataframe` to get a DataFrame for each file, and adds a `SOURCE_FILE_INDEX` column. This index is critical for maintaining the processing order of your files (since later files represent more recent tests).
    * **Concatenation:** All individual DataFrames are concatenated into a single large DataFrame (`merged_df`).
    * **Sorting:** This is the most crucial step for deduplication. The `merged_df` is sorted by:
        * `HEAD_NUM`, `SITE_NUM`: To group all tests for a specific physical part. If your STDF files contain `PART_ID` in `PRR` records and you can link it to `PTR` records, using `PART_ID` would be even more robust.
        * `TEST_NUM`, `TEST_NAM`: To identify a specific test within a part's test sequence.
        * `SOURCE_FILE_INDEX`: This ensures that results from later STDF files (which contain re-tests) appear *after* results for the same part/test from earlier files.
        * `RTN_ICNT`: If a part/test was re-tested *within* the same STDF file (less common for your scenario, but good practice), a higher `RTN_ICNT` would indicate the later retest.
    * **Deduplication:** `merged_df.drop_duplicates(subset=[...], keep='last')` is used.
        * `subset`: Defines the columns that uniquely identify a "test for a part" (e.g., `HEAD_NUM`, `SITE_NUM`, `TEST_NUM`, `TEST_NAM`).
        * `keep='last'`: Because of the careful sorting, `keep='last'` will retain the entry that was processed *last* for each unique part-test combination. This means it will keep the result from the latest STDF file and the highest `RTN_ICNT` if there were multiple retests for that specific test within a single file or across files.
    * **Saving:** The `final_results_df` is then saved to a CSV file.

**Important Considerations and Potential Enhancements:**

* **Part Identification:** The current example uses `HEAD_NUM` and `SITE_NUM` to identify a part. In a real-world scenario, you will very likely need to parse `PRR` (Part Results Record) records to get the unique `PART_ID` and potentially `WAFER_ID` if tests are conducted at the wafer level. Then, you'd need to associate the `PTR` records with their corresponding `PRR` (usually by `HEAD_NUM` and `SITE_NUM` and order of appearance in the file) to get the complete part identification.
* **Test Identification:** The combination of `TEST_NUM` and `TEST_NAM` is generally good for identifying a unique test.
* **Retest Logic (RTN_ICNT):** The `RTN_ICNT` field in `PTR` records is crucial. If a part failed and was re-tested, `RTN_ICNT` will increment. Sorting by this helps ensure the latest result is kept even if multiple retests appear in the same file.
* **Other Record Types:** Depending on your analysis needs, you might want to extract information from other STDF record types (e.g., `PIR` for Part Information, `TSR` for Test Summary, `HBR` for Hardware Bin, `SBR` for Software Bin). The `Semi-ATE` library provides access to all these.
* **Error Handling:** The `parse_stdf_to_dataframe` includes basic error handling, but for production code, you might want more robust logging and error management.
* **Memory Usage:** For very large STDF files or many files, loading all data into a single pandas DataFrame might consume significant memory. If this becomes an issue, consider:
    * **Processing in chunks:** Process records in smaller batches if the `Semi-ATE` library supports it for `PTR` records.
    * **Database:** Load data into a temporary database (like SQLite) and perform the merge/deduplication with SQL queries, which are optimized for large datasets.
    * **Parquet/HDF5:** Save intermediate DataFrames to Parquet or HDF5 format, which are more efficient for columnar data storage and retrieval, and then load/merge.
* **Custom Retest Definitions:** Your specific retest methodology might have nuances. For example, if a part is sent to a *different* tester for retest, or if the "final" result is determined by something other than the latest timestamp or retest count, you'll need to adjust the sorting and deduplication logic accordingly.

This code provides a solid foundation for merging your STDF files and extracting the final test results. Remember to adapt the part and test identification logic to precisely match how these are recorded in your specific STDF files.